In [10]:
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf

# 1. SPY 옵션 체인 데이터 로드
data = pd.read_csv('spy_2020_2022.csv')

C:\Users\young\AppData\Local\Temp\ipykernel_14092\4060673812.py:6: DtypeWarning: Columns (8,9,10,11,12,13,15,17,18,20,21,23,24,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('spy_2020_2022.csv')


In [ ]:
# 2. 컬럼명 정리 (공백, 대괄호 제거)
data.columns = data.columns.str.strip().str.replace('[', '').str.replace(']', '')

# 3. QUOTE_DATE를 datetime 타입으로 변환
data['QUOTE_DATE'] = pd.to_datetime(data['QUOTE_DATE'])

# 4. 무위험이자율 (r) 가져오기
r_data = web.DataReader('DGS1', 'fred', start='2020-01-01', end='2022-12-31')
r_data = r_data.fillna(method='ffill')
r_data.index = pd.to_datetime(r_data.index)

# 5. QUOTE_DATE 기준 merge
data = data.merge(r_data, left_on='QUOTE_DATE', right_index=True, how='left')
data = data.rename(columns={'DGS1': 'r'})  # r이라는 컬럼명으로 통일

# 6. 배당수익률 (q) 가져오기
spy = yf.Ticker("SPY")
div_yield = spy.info['dividendYield']

# 7. 모든 행에 배당수익률(q) 추가
data['q'] = div_yield

# 8. 결과 확인
print(data[['QUOTE_DATE', 'r', 'q']].head())

# 9. (선택) 저장
data.to_csv('spy_2020_2022_with_rq.csv', index=False)

C:\Users\young\AppData\Local\Temp\ipykernel_14092\3686543891.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  r_data = r_data.fillna(method='ffill')


  QUOTE_DATE     r     q
0 2021-09-01  0.07  1.28
1 2021-09-01  0.07  1.28
2 2021-09-01  0.07  1.28
3 2021-09-01  0.07  1.28
4 2021-09-01  0.07  1.28
